# Map pesticide and drug SMILES to DTXSID

In [14]:
import polars as pl
import pandas as pd
import math
from standardize_smiles import StandardizeMolecule

In [5]:
data_dir = "/data"

In [ ]:
# Read in compound lists
pesticides = pd.read_csv(f"../{data_dir}/pesticides.csv")
drugs = pd.read_csv(f"../{data_dir}/pesticides.csv")

In [9]:
# Standardize the smiles
standardized_pesticides = StandardizeMolecule(input=pesticides, augment=True).run()
standardized_drugs = StandardizeMolecule(input=drugs, augment=True).run()

100%|██████████| 993/993 [00:17<00:00, 56.34it/s] 


In [ ]:
# Process RefChemDB
refchemdb = pl.read_csv(f"../{data_dir}/refchemdb/refchemdb.csv").select(
    'dsstox_substance_id'
).unique()

chunk_size = 10_000
total_rows = refchemdb.height
num_chunks = math.ceil(total_rows / chunk_size)

for i in range(num_chunks):
    start = i * chunk_size
    end = min(start + chunk_size, total_rows)
    chunk = refchemdb.slice(start, end - start)
    chunk.write_csv(f"../{data_dir}/refchemdb/input_comptoxdb/refchemdb_dtxsid_{i+1}.csv")